<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Criminology/blob/main/ACS_2010_2022_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd

all_dfs = []

for year in range(2010, 2023):
  if year != 2020:
    df = pd.read_csv(f"{year}_State.csv")

    df = df.loc[:, df.columns.str.contains("!!Estimate") | (df.columns == "Label (Grouping)")]
    df.columns = df.columns.str.replace("!!Total population!!Estimate", "")

    df = df.iloc[[19, 8, 105, 173, 255], :]

    df_t = df.transpose().reset_index()
    df_t.columns = df_t.iloc[0, :]
    df_t = df_t.iloc[1:, :]

    new_column_names = ["Label (Grouping)", "Median age years",	"Percent Male"	,"Percent BA or higher"	,"Percent unemployed",	"Per capita income dollars"]
    df_t.columns = new_column_names

    df_t["Year"] = year

    all_dfs.append(df_t)

df_state = pd.concat(all_dfs, ignore_index=True)

import pandas as pd

df_state.iloc[:, 2:] = df_state.iloc[:, 2:].applymap(lambda x: str(x).replace("%", "").replace(",", ""))

df_state.iloc[:, 1:] = df_state.iloc[:, 1:].apply(lambda x: pd.to_numeric(x.replace('[^0-9.]', '', regex=True), errors='coerce'))

df_state

<ipython-input-85-4607051d5ce8>:31: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_state.iloc[:, 1:] = df_state.iloc[:, 1:].apply(lambda x: pd.to_numeric(x.replace('[^0-9.]', '', regex=True), errors='coerce'))


,Label (Grouping),Median age years,Percent Male,Percent BA or higher,Percent unemployed,Per capita income dollars,Year
0,Alabama,37.8,48.5,21.9,7.0,21993.0,2010
1,Alaska,33.8,52.2,27.9,6.6,30598.0,2010
2,Arizona,35.9,49.7,25.9,7.2,23618.0,2010
3,Arkansas,37.3,49.0,19.5,5.8,20725.0,2010
4,California,35.2,49.7,30.1,8.2,27353.0,2010
...,...,...,...,...,...,...,...
608,Virginia,39.0,49.5,42.2,2.3,47199.0,2022
609,Washington,38.4,50.5,39.5,2.6,50764.0,2022
610,West Virginia,42.9,49.7,24.8,2.5,31922.0,2022
611,Wisconsin,40.4,50.2,33.2,1.8,40188.0,2022


In [86]:
df_temp = df_state[df_state["Year"].isin([2019, 2021])]

df_temp.columns

df_temp_2020 = df_temp.groupby("Label (Grouping)").mean().reset_index()

df_temp_2020

df_state = pd.concat([df_state, df_temp_2020], ignore_index=True)

df_state["Year"].value_counts()

2011.0    52
2010.0    51
2012.0    51
2013.0    51
2014.0    51
2015.0    51
2016.0    51
2017.0    51
2018.0    51
2019.0    51
2021.0    51
2022.0    51
2020.0    51
Name: Year, dtype: int64

In [87]:
# Race

all_dfs = []

import pandas as pd

for year in range(2010, 2023):
  df = pd.read_csv(f"{year}_Race.csv")

  df = df[~df["Label (Grouping)"].str.contains("Error")]

  df.fillna(method='bfill', inplace=True)
  df = df.drop_duplicates(subset = ["Total:"], keep="first")

  df = df.iloc[:, :3]

  df.iloc[:, 1:] = df.iloc[:, 1:].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce'))
  df["Percent White"] = df.iloc[:, 2] / df.iloc[:, 1] * 100
  df["Percent White"] = df["Percent White"].apply(pd.to_numeric)
  df["Percent White"] = round(df["Percent White"], 2)

  df["Year"] = year

  all_dfs.append(df)

df_result = pd.concat(all_dfs, ignore_index=True)

df_result = df_result[["Label (Grouping)", "Year", "Percent White"]]

df_White = df_result
df_White

,Label (Grouping),Year,Percent White
0,Alabama,2010,69.47
1,Alaska,2010,67.43
2,Arizona,2010,79.44
3,Arkansas,2010,78.37
4,California,2010,62.45
...,...,...,...
671,Washington,2022,65.88
672,West Virginia,2022,90.32
673,Wisconsin,2022,80.40
674,Wyoming,2022,84.57


In [90]:
print(df_state.columns)
print(df_White.columns)

Index(['Label (Grouping)', 'Median age years', 'Percent Male',
       'Percent BA or higher', 'Percent unemployed',
       'Per capita income dollars', 'Year'],
      dtype='object')
Index(['Label (Grouping)', 'Year', 'Percent White'], dtype='object')


In [95]:
df_1022 = pd.merge(df_White, df_state, on=["Label (Grouping)", "Year"])


df_1022

,Label (Grouping),Year,Percent White,Median age years,Percent Male,Percent BA or higher,Percent unemployed,Per capita income dollars
0,Alabama,2010,69.47,37.8,48.5,21.9,7.0,21993.0
1,Alaska,2010,67.43,33.8,52.2,27.9,6.6,30598.0
2,Arizona,2010,79.44,35.9,49.7,25.9,7.2,23618.0
3,Arkansas,2010,78.37,37.3,49.0,19.5,5.8,20725.0
4,California,2010,62.45,35.2,49.7,30.1,8.2,27353.0
...,...,...,...,...,...,...,...,...
659,Virginia,2022,60.18,39.0,49.5,42.2,2.3,47199.0
660,Washington,2022,65.88,38.4,50.5,39.5,2.6,50764.0
661,West Virginia,2022,90.32,42.9,49.7,24.8,2.5,31922.0
662,Wisconsin,2022,80.40,40.4,50.2,33.2,1.8,40188.0


In [96]:
! pip install us
import us

def state_name_to_abbreviation(state_name):
    state = us.states.lookup(state_name)
    if state:
        return state.abbr
    else:
        return state_name

df_1022['State'] = df_1022["Label (Grouping)"].apply(state_name_to_abbreviation)

df_1022["State"] = df_1022["State"].str.replace("District of Columbia", "DC")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.2 MB/s eta 0:00:00
  Created wheel for us: filename=us-3.1.1-py3-none-any.whl size=12547 sha256=75ad7e8d28a21660a43f9b0f1ead3c55dc807555812b5487571a7e881c144530
  Stored in directory: /root/.cache/pip/wheels/da/f9/64/9669135a7ce533faf78e3b32259c505f964f03b069be6844f8
Successfully built us


In [99]:
df_1022.rename(columns={"Label (Grouping)":"Geographic Name"}, inplace=True)


,Geographic Name,Year,Percent White,Median age years,Percent Male,Percent BA or higher,Percent unemployed,Per capita income dollars,State
0,Alabama,2010,69.47,37.8,48.5,21.9,7.0,21993.0,AL
1,Alaska,2010,67.43,33.8,52.2,27.9,6.6,30598.0,AK
2,Arizona,2010,79.44,35.9,49.7,25.9,7.2,23618.0,AZ
3,Arkansas,2010,78.37,37.3,49.0,19.5,5.8,20725.0,AR
4,California,2010,62.45,35.2,49.7,30.1,8.2,27353.0,CA
...,...,...,...,...,...,...,...,...,...
659,Virginia,2022,60.18,39.0,49.5,42.2,2.3,47199.0,VA
660,Washington,2022,65.88,38.4,50.5,39.5,2.6,50764.0,WA
661,West Virginia,2022,90.32,42.9,49.7,24.8,2.5,31922.0,WV
662,Wisconsin,2022,80.40,40.4,50.2,33.2,1.8,40188.0,WI


In [100]:
df_1022 = df_1022[["State", "Geographic Name", "Year", "Median age years", "Percent Male", "Percent White", "Percent BA or higher", "Percent unemployed", "Per capita income dollars"]]

In [102]:
df_1022.to_csv("ACS_2010_2022.csv", index=False)

# Test the file

In [103]:
! pip install plotly
! pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [109]:

import plotly.express as px
import kaleido

fig = px.choropleth(df_1022,
                    locations="State",
                    locationmode='USA-states',
                    color="Percent White",
                    range_color = [30,80],
                    color_continuous_scale="Blues",
                    scope='usa',
                    animation_frame='Year',
                    title = "Percent White Over States")

fig.write_html("Percent White.html")
fig.show()